In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from rozumarm_vima_utils.scene_renderer import VIMASceneRenderer
from rozumarm_vima_utils.transform import rf_tf_c2r, map_tf_repr_c2r

In [ ]:
N_SWEPT_OBJECTS = 2
r = VIMASceneRenderer('sweep_without_exceeding')
import sys
sys.path.extend([
    "/home/daniil/code/rozumarm-vima/rozumarm_vima/rozumarm_vima_cv",
    "/home/daniil/code/rozumarm-vima/rozumarm_vima/utils",
    "/home/daniil/code/rozumarm-vima/rozumarm_vima/camera_utils"
])

In [ ]:
from rozumarm_vima.detectors import detector

In [ ]:
r.reset(exact_num_swept_objects=N_SWEPT_OBJECTS)

n_cubes = -1
while n_cubes != 2 * N_SWEPT_OBJECTS:
    obj_posquats = detector.detect()
    n_cubes = len(obj_posquats)

# map from cam to rozum
obj_posquats = [
    (rf_tf_c2r(pos), map_tf_repr_c2r(quat)) for pos, quat in obj_posquats
]

front_img, top_img = r.render_scene(obj_posquats)

In [ ]:
obs = r.env._get_obs()
plt.imshow(np.transpose(obs['rgb']['top'], axes=(1, 2, 0)))

### checking cube texture mapping

In [ ]:
import matplotlib.pyplot as plt
from rozumarm_vima_utils.scene_renderer import VIMASceneRenderer
from rozumarm_vima_utils.transform import rf_tf_c2r, map_tf_repr_c2r
from rozumarm_vima.detectors import detector
cubes_detector = detector

N_SWEPT_OBJECTS = 2

In [ ]:
def prepare_sim_obs(detector, env_renderer):
    n_cubes = -1
    while n_cubes != 2 * N_SWEPT_OBJECTS:
        obj_posquats = detector.detect()
        n_cubes = len(obj_posquats)

    # map from cam to rozum
    obj_posquats = [
        (rf_tf_c2r(pos), map_tf_repr_c2r(quat)) for pos, quat in obj_posquats
    ]

    env_renderer.render_scene(obj_posquats)
    obs, *_ = env_renderer.env.step(action=None)

    _, top_cam_image = detector.cam_1.read_image()
    _, front_cam_image = detector.cam_2.read_image()
    return obs, top_cam_image, front_cam_image

In [ ]:
r = VIMASceneRenderer('sweep_without_exceeding')
oracle = r.env.task.oracle(r.env)

In [ ]:
n_episodes = 5

real_images = []
sim_images = []
sim_images_after_swipes = []

for i in range(n_episodes):
    r.reset(exact_num_swept_objects=N_SWEPT_OBJECTS)
    obs, top_cam_image, front_cam_image = prepare_sim_obs(cubes_detector, r)
    real_images.append(top_cam_image)
    sim_images.append(obs["rgb"]["top"])

    for _ in range(2):
        action = oracle.act(obs)
        next_obs, *_ = r.env.step(action)
    sim_images_after_swipes.append(next_obs["rgb"]["top"])

In [ ]:
fig, axs = plt.subplots(n_episodes, 3, figsize=(12, 16))
fig.subplots_adjust(wspace=0.1, hspace=0.1)

for i in range(n_episodes):
    axs[i, 0].axis('off')
    axs[i, 0].imshow(real_images[i][::-1])

    axs[i, 1].axis('off')
    axs[i, 1].imshow(sim_images[i].transpose(1, 2, 0))

    axs[i, 2].axis('off')
    axs[i, 2].imshow(sim_images_after_swipes[i].transpose(1, 2, 0))


axs[0, 0].set_title("real camera")

pad = 35
axs[0, 1].set_title("sim camera, before swipes", pad=pad)
axs[0, 2].set_title("sim camera, after swipes", pad=pad)

fig.suptitle("The test of texture consistency over 5 episodes (shown as rows).", y=0.925)

In [ ]:
fig.savefig('cubes_texture_mapping.jpg', dpi=600, bbox_inches='tight')